In [1]:
from ultralytics import YOLO
import os
import gdown
import shutil
import yaml

### Вспомогательные функции

In [3]:
# Загрузить разметку с гугл диска
def load_pseudo_labels(
        dataset_name, #VOC
        ann_name, # Название разметки
        ann_postfix, # Постфикс разметки
        url # Урл, чтобы скачать (google disk)
        ):
    
    save_dir = f'../data/pseudo_labels/{dataset_name}'
    os.makedirs(save_dir, exist_ok=True)
    # Загрузка разметки
    filename = f'{save_dir}/{ann_name}.zip'
    gdown.download(url, filename, quiet = True, fuzzy = True)
    !unzip -d {save_dir} {filename} 

    # Копирование в папку с датасетом
    for sample in (['train', 'test', 'val']):
        lbs_source = f'{save_dir}/{ann_name}/{sample}'
        if os.path.exists(lbs_source) is False:
            print(f'{lbs_source} does not exisist')
            continue
        lbs_dest = f'../datasets/{dataset_name}/labels/{sample}_{ann_postfix}'
        shutil.copytree(lbs_source, lbs_dest)

# Подгрузить разметку в зависимости от эксперимента
def prepare_labels(dataset_name, # Название датасета
                   train_postfix=None, # Разметка, по умолчанию оригинальная [orig, GD]
                   val_postfix=None, # Разметка, по умолчанию оригинальная [orig, GD]
                   test_postfix=None # Разметка, по умолчанию оригинальная [orig, GD]
                   ):
    # Перезаписать разметку на разметку на выбранную
    for sample, postfix in zip(['train', 'val', 'test'], [train_postfix, val_postfix, test_postfix]):
        if postfix is None:
            postfix = 'orig'

        lbs_dest = f'../datasets/{dataset_name}/labels/{sample}'
        lbs_source = f'../datasets/{dataset_name}/labels/{sample}_{postfix}'
        !rm -rf {lbs_dest}
        shutil.copytree(lbs_source, lbs_dest)

# Чтение конфигурационного файла
def read_yaml_file(file_path):
    with open(file_path, 'r') as file:
        try:
            data = yaml.safe_load(file)
            return data
        except yaml.YAMLError as e:
            print("Error reading YAML file:", e)

### Загрузка разметки

In [3]:
# Загрузить разметку с гугл диска
# load_pseudo_labels(
#         dataset_name='VOC', #VOC
#         ann_name='annotations_nms_0.7_p_0.4', # Название разметки
#         ann_postfix='GD', # Постфикс разметки
#         url='https://drive.google.com/file/d/1K9-6IppbpJTnLivh80Or4Oj5tZUgd3ej/view?usp=sharing' # Урл, чтобы скачать (google disk)
#         )


### Запуск экспериментов

In [6]:
!ls runs/detect/orig_yolov8n_train

args.yaml			 PR_curve.png	   val_batch0_labels.jpg
confusion_matrix_normalized.png  R_curve.png	   val_batch0_pred.jpg
confusion_matrix.png		 results.csv	   val_batch1_labels.jpg
F1_curve.png			 results.png	   val_batch1_pred.jpg
labels_correlogram.jpg		 train_batch0.jpg  val_batch2_labels.jpg
labels.jpg			 train_batch1.jpg  val_batch2_pred.jpg
P_curve.png			 train_batch2.jpg  weights


In [8]:
%cd ..

/home/nikolay/Documents/Projects/Diploma/My repos/Pseudo_label_YOLO


/home/nikolay/Documents/Projects/Diploma/My repos/Pseudo_label_YOLO/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [10]:
#test
weight_path = 'notebooks/runs/detect/orig_yolov8n_train/weights/best.pt'
model =YOLO(weight_path)
validation_results = model.val(data='datasets/VOC/VOC_test.yaml', imgsz=640, batch=10)

Ultralytics YOLOv8.1.35 🚀 Python-3.10.6 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5938MiB)
Model summary (fused): 168 layers, 3009548 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/nikolay/Documents/Projects/Diploma/My repos/Pseudo_label_YOLO/datasets/VOC/labels/test.cache... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 496/496 [00:22<00:00, 22.43it/s]


                   all       4952      12032      0.731      0.425      0.542      0.367
             aeroplane       4952        285       0.72      0.515      0.596      0.416
               bicycle       4952        337      0.658      0.608       0.65      0.425
                  bird       4952        459      0.723      0.414      0.507      0.322
                  boat       4952        263       0.81     0.0164      0.327      0.185
                bottle       4952        469      0.345     0.0405      0.112      0.078
                   bus       4952        213          1     0.0119      0.121     0.0887
                   car       4952       1201      0.656      0.734      0.755      0.553
                   cat       4952        358      0.567      0.768      0.717      0.464
                 chair       4952        756      0.535      0.437      0.477      0.306
                   cow       4952        244      0.733      0.316      0.466      0.356
           diningtabl

In [14]:
validation_results.save_dir

PosixPath('runs/detect/val2')

In [12]:
type(validation_results)

ultralytics.utils.metrics.DetMetrics

In [13]:
import pickle

In [16]:
!ls

configs  datasets   README.md	      runs    yolov8n.pt
data	 notebooks  requirements.txt  source


In [19]:
import torch

In [20]:
torch.save(validation_results, 'model_state_dict.pth')

TypeError: cannot pickle 'torch._C.Generator' object

In [36]:
# Save the file
pickle.dump(validation_results.box, file = open(f'{str(validation_results.save_dir)}/results.pickle', "wb"))
# Reload the file
# company1_reloaded = pickle.load(open("company1.pickle", "rb"))

In [37]:
reloaded = pickle.load(open(f'{str(validation_results.save_dir)}/results.pickle', "rb"))

In [41]:
reloaded.ap50

array([    0.59646,     0.64999,     0.50722,     0.32681,     0.11159,     0.12132,     0.75523,     0.71697,     0.47709,     0.46577,      0.5214,     0.59464,     0.77223,     0.75525,     0.78346,     0.20647,     0.65879,     0.53316,     0.69266,     0.60043])

In [35]:
# metrics.box.map    # map50-95
# metrics.box.map50  # map50
# metrics.box.map75  # map75
# metrics.box.maps 
validation_results.box.all_ap[:,-1]

array([   0.027735,    0.012984,   0.0093837,   0.0024506,   0.0018597,   0.0058105,    0.035251,    0.024297,     0.01992,    0.037053,     0.00579,    0.033996,    0.047395,   0.0098933,    0.055887,   0.0019691,    0.025987,    0.096861,   0.0062307,    0.016254])